In [10]:
import re
import itertools as it
import random as rm

In [23]:
goals = [("G1","dd"),("G2","ht"),("G3","sd"),("G4","cb")]
for g,s in goals:
    print([f"c({s},{i})" for i in range(5,0,-1)])

['c(dd,5)', 'c(dd,4)', 'c(dd,3)', 'c(dd,2)', 'c(dd,1)']
['c(ht,5)', 'c(ht,4)', 'c(ht,3)', 'c(ht,2)', 'c(ht,1)']
['c(sd,5)', 'c(sd,4)', 'c(sd,3)', 'c(sd,2)', 'c(sd,1)']
['c(cb,5)', 'c(cb,4)', 'c(cb,3)', 'c(cb,2)', 'c(cb,1)']


# Generate game setups

In [59]:
clrs = ["dd","ht","sd","cb"]
rnks = range(1,14)

In [60]:
cards = list(map(lambda rc: f"c({rc[0]},{rc[1]})", it.product(rnks,clrs)))

In [61]:
cards = rm.sample(cards, k=len(cards))

In [62]:
l = [[],[],[],[],[],[],[],[]]

In [63]:
for i in range(len(cards)):
    l[i%8].append(cards[i])

In [66]:
print(l)

[['c(11,sd)', 'c(11,ht)', 'c(13,cb)', 'c(1,dd)', 'c(1,sd)', 'c(9,cb)', 'c(4,sd)'], ['c(12,ht)', 'c(5,ht)', 'c(13,dd)', 'c(12,sd)', 'c(4,cb)', 'c(9,ht)', 'c(3,dd)'], ['c(10,sd)', 'c(8,sd)', 'c(5,sd)', 'c(8,dd)', 'c(1,ht)', 'c(2,ht)', 'c(3,ht)'], ['c(2,dd)', 'c(10,cb)', 'c(12,cb)', 'c(2,cb)', 'c(3,sd)', 'c(6,sd)', 'c(5,dd)'], ['c(6,ht)', 'c(10,dd)', 'c(6,dd)', 'c(8,cb)', 'c(8,ht)', 'c(11,cb)'], ['c(12,dd)', 'c(7,cb)', 'c(1,cb)', 'c(2,sd)', 'c(7,dd)', 'c(3,cb)'], ['c(5,cb)', 'c(6,cb)', 'c(4,dd)', 'c(10,ht)', 'c(4,ht)', 'c(13,ht)'], ['c(13,sd)', 'c(7,ht)', 'c(9,sd)', 'c(7,sd)', 'c(9,dd)', 'c(11,dd)']]


In [67]:
cmd = f"fc({','.join(str(l[i]) for i in range(8))}, S)".replace("'","")

In [68]:
print(cmd)

fc([c(11,sd), c(11,ht), c(13,cb), c(1,dd), c(1,sd), c(9,cb), c(4,sd)],[c(12,ht), c(5,ht), c(13,dd), c(12,sd), c(4,cb), c(9,ht), c(3,dd)],[c(10,sd), c(8,sd), c(5,sd), c(8,dd), c(1,ht), c(2,ht), c(3,ht)],[c(2,dd), c(10,cb), c(12,cb), c(2,cb), c(3,sd), c(6,sd), c(5,dd)],[c(6,ht), c(10,dd), c(6,dd), c(8,cb), c(8,ht), c(11,cb)],[c(12,dd), c(7,cb), c(1,cb), c(2,sd), c(7,dd), c(3,cb)],[c(5,cb), c(6,cb), c(4,dd), c(10,ht), c(4,ht), c(13,ht)],[c(13,sd), c(7,ht), c(9,sd), c(7,sd), c(9,dd), c(11,dd)], S)


In [69]:
cmd

'fc([c(11,sd), c(11,ht), c(13,cb), c(1,dd), c(1,sd), c(9,cb), c(4,sd)],[c(12,ht), c(5,ht), c(13,dd), c(12,sd), c(4,cb), c(9,ht), c(3,dd)],[c(10,sd), c(8,sd), c(5,sd), c(8,dd), c(1,ht), c(2,ht), c(3,ht)],[c(2,dd), c(10,cb), c(12,cb), c(2,cb), c(3,sd), c(6,sd), c(5,dd)],[c(6,ht), c(10,dd), c(6,dd), c(8,cb), c(8,ht), c(11,cb)],[c(12,dd), c(7,cb), c(1,cb), c(2,sd), c(7,dd), c(3,cb)],[c(5,cb), c(6,cb), c(4,dd), c(10,ht), c(4,ht), c(13,ht)],[c(13,sd), c(7,ht), c(9,sd), c(7,sd), c(9,dd), c(11,dd)], S)'

In [70]:
print(f"swipl -g '{cmd},write(S)' -t halt freecell.pl > solution_1.txt")

swipl -g 'fc([c(11,sd), c(11,ht), c(13,cb), c(1,dd), c(1,sd), c(9,cb), c(4,sd)],[c(12,ht), c(5,ht), c(13,dd), c(12,sd), c(4,cb), c(9,ht), c(3,dd)],[c(10,sd), c(8,sd), c(5,sd), c(8,dd), c(1,ht), c(2,ht), c(3,ht)],[c(2,dd), c(10,cb), c(12,cb), c(2,cb), c(3,sd), c(6,sd), c(5,dd)],[c(6,ht), c(10,dd), c(6,dd), c(8,cb), c(8,ht), c(11,cb)],[c(12,dd), c(7,cb), c(1,cb), c(2,sd), c(7,dd), c(3,cb)],[c(5,cb), c(6,cb), c(4,dd), c(10,ht), c(4,ht), c(13,ht)],[c(13,sd), c(7,ht), c(9,sd), c(7,sd), c(9,dd), c(11,dd)], S),write(S)' -t halt freecell.pl > solution_1.txt


In [20]:
print(cmd.replace("'",""))

fc([c(3,dd), c(4,ht), c(1,cb)],[c(3,cb), c(4,dd), c(5,cb)],[c(1,dd), c(5,dd), c(2,cb)],[c(5,sd), c(3,ht), c(1,sd)],[c(4,sd), c(2,sd)],[c(2,dd), c(4,cb)],[c(2,ht), c(3,sd)],[c(5,ht), c(1,ht)], S)
